# [Module 4.2] getRec() 통한 추천 리스트 얻기

이 노트북은 Module2에서 생성한 솔류션을 바탕으로 아래와 같은 작업을 합니다.
* 캠페인 생성
* 캠페인을 통해 특정 유저에 대한 추천 영화 리스트 얻기


In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 샘플 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

In [4]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()

items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
items.tail()

#len(unique_items)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,ITEM_ID,TITLE,GENRE
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [9]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

def get_movie_genre(movie_id):
    """
    Takes in an ID, returns a genre
    """
    movie_id = int(movie_id)
    movie_genre=items[items['ITEM_ID']==movie_id]['GENRE']
    return (movie_genre.tolist())


#### HRNN GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [26]:
# Help 함수
def rec_item_list(item_list):
    recommendation_title_list = []
    recommendation_genre_list = []
    # recommendation_id_list=[]
    for item in item_list:
        title = get_movie_title(item['itemId'])
        genre = get_movie_genre(item['itemId'])
        recommendation_title_list.append(title)
        recommendation_genre_list.append(genre)    
        
    recommendations_df = pd.DataFrame(data={'OriginalRecs':recommendation_title_list,
                                            'Genre': recommendation_genre_list})
    return recommendations_df                                                

In [27]:
df=pd.read_csv(warm_train_interaction_filename)

# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
print("USER: {}".format(user_id))

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendations_df = rec_item_list(item_list)
recommendations_df

USER: 3415
Recommendations for user:  3415


,OriginalRecs,Genre
0,[Vacation (1983)],[Comedy]
1,[Ruthless People (1986)],[Comedy]
2,[Who Framed Roger Rabbit? (1988)],[Adventure|Animation|Film-Noir]
3,[Real Genius (1985)],[Comedy]
4,[Lethal Weapon (1987)],[Action|Comedy|Crime|Drama]
5,"[Good Morning, Vietnam (1987)]",[Comedy|Drama|War]
6,[Batman (1989)],[Action|Adventure|Crime|Drama]
7,[Indiana Jones and the Temple of Doom (1984)],[Action|Adventure]
8,[Fatal Attraction (1987)],[Thriller]
9,[Better Off Dead... (1985)],[Comedy]


#### Sims GetRecommendations 호출
아래 코드 셀을 실행하면 특정 아이템과 유사한 추천 영화 목록이 반환됩니다.

In [41]:
# Getting a random item_id:
_, item_id, _,_,_ = df.sample().values[0]
title = get_movie_title(item_id)
genre = get_movie_genre(item_id)

print("Recommendations for item_id: {}, Title:{}, Genre:{}".format(item_id,title[0], genre[0]))

# print("Movie title: {}, Genre:{} ".format(title[0], genre[0]))


get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(item_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)



item_list = get_recommendations_response['itemList']
recommendations_df = rec_item_list(item_list)
recommendations_df


Recommendations for item_id: 3261, Title:Singles (1992), Genre:Comedy|Drama|Romance


,OriginalRecs,Genre
0,[Benny & Joon (1993)],[Comedy|Romance]
1,[To Die For (1995)],[Comedy|Drama]
2,[So I Married an Axe Murderer (1993)],[Comedy|Romance|Thriller]
3,[Honeymoon in Vegas (1992)],[Comedy|Romance]
4,"[Truth About Cats & Dogs, The (1996)]",[Comedy|Romance]
5,"[House of Yes, The (1997)]",[Comedy|Drama|Thriller]
6,[L.A. Story (1991)],[Comedy|Romance]
7,[Pretty Woman (1990)],[Comedy|Romance]
8,[While You Were Sleeping (1995)],[Comedy|Romance]
9,[Dazed and Confused (1993)],[Comedy]



## Personalized Ranking

Personalized Ranking의 핵심 사용 사례는 아이템 리스트를 가져 와서 사용자에게 우선 순위 또는 사용자 관심 순서로 표시하는 것입니다. 이 기능에 대해 알아보기 위해 이번 파트에서는 한명의 사용자와 랜덤하게 뽑은 25개의 아이템 항목을 가지고 테스트 해 봅니다. 

In [ ]:
#Get the user list
df=pd.read_csv(interaction_filename)
df_users = df['USER_ID'].unique()
df_users=pd.DataFrame(df_users,columns=['USER_ID'])
df_items=df['ITEM_ID'].unique()
df_items=pd.DataFrame(df_items,columns=['ITEM_ID'])

rerank_user = df_users['USER_ID'].sample(1).tolist()[0]
rerank_items = df_items['ITEM_ID'].sample(25).tolist()

In [ ]:
rerank_list = []
for item in rerank_items:
    title = get_movie_title(item)
    rerank_list.append(title)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df


In [ ]:
# Convert user to string:
user_id = str(rerank_user)

rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = ranking_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    title = get_movie_title(item['itemId'])
    ranked_list.append(title)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

## 리뷰

캠페인을 생성하고 실제적으로 특정 유저의 추천 영화 목록도 얻었습니다.
이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`4.View_Campaign_And_Interactions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [ ]:
%store recommendations_df
%store user_id